In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json
from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                data = {'search': input_.loc[a, 'Part_Number'],
                        'page': '1'}

                resp = requests.post('https://uacparts.com/SearchAjaxList/Search',
                                     data=data,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =

                json_data = json.loads(resp)['d']['c'].strip()
                soup = BeautifulSoup(json_data, 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =
                
                index_part = -1
                list_part_number = html.xpath('//h2[@class="part-label"]/text()')
                for i in range(len(list_part_number)):
                    if list_part_number[i].replace(' ', '') == input_.loc[a, 'Part_Number']:
                        index_part = i
                        
                if index_part == -1:
                    continue

                # = = = = = = = = = = = = = = =
                
                src = html.xpath('//img[@alt="picture"]/@src')[index_part].strip()

                # = = = = = = = = = = = = = = =
                
                list_div = html.xpath('//div[@class="part-info"]/div[@class="comments"]')[index_part*4:index_part*4+5]
                list_div = [div.xpath('./text()')[0] if len(div.xpath('./text()')) != 0 else '' for div in list_div]

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                                         'UAC_Part_Number': list_part_number[index_part],
                                         'UAC_1': '',
                                         'UAC_2': '',
                                         'UAC_3': '',
                                         'UAC_4': '',
                                         'UAC_Vehicle_1': '',
                                         'UAC_Vehicle_2': '',
                                         'UAC_OE_Four_Seasons': '',
                                         'UAC_OE_Omega': '',
                                         'UAC_OE': '',
                                         'UAC_Pic': '',
                                         'UAC_Src': ''}])

                for i in range(4):
                    df_temp.loc[0, 'UAC_' + str(i+1)] = list_div[i].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Part_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./uac_part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./uac_part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1156

[ok] - CO101210C
[尝试次数：1] - [剩余数量：1136] - [当前时间：18:15:34]

[ok] - CO0076C
[尝试次数：1] - [剩余数量：1135] - [当前时间：18:15:34]

[ok] - CO0023GLC
[尝试次数：1] - [剩余数量：1134] - [当前时间：18:15:34]

[ok] - CO10082JC
[尝试次数：1] - [剩余数量：1133] - [当前时间：18:15:35]

[ok] - CO0041C
[尝试次数：1] - [剩余数量：1132] - [当前时间：18:15:35]

[ok] - CO10076C
[尝试次数：1] - [剩余数量：1131] - [当前时间：18:15:35]

[ok] - CO0074C
[尝试次数：1] - [剩余数量：1130] - [当前时间：18:15:35]

[ok] - CO10120C
[尝试次数：1] - [剩余数量：1129] - [当前时间：18:15:35]

[ok] - CO10016C
[尝试次数：1] - [剩余数量：1128] - [当前时间：18:15:35]

[ok] - CO04080C
[尝试次数：1] - [剩余数量：1127] - [当前时间：18:15:35]

[ok] - CO0020GLC
[尝试次数：1] - [剩余数量：1126] - [当前时间：18:15:35]

[ok] - CO0040C
[尝试次数：1] - [剩余数量：1125] - [当前时间：18:15:35]

[ok] - CO0011C
[尝试次数：1] - [剩余数量：1124] - [当前时间：18:15:35]

[ok] - CO0104JC
[尝试次数：1] - [剩余数量：1123] - [当前时间：18:15:36]

[ok] - CO101200C
[尝试次数：1] - [剩余数量：1122] - [当前时间：18:15:36]

[ok] - CO101220C
[尝试次数：1] - [剩余数量：1121] - [当前时间：18:15:36]

[ok] - CO101230C
[尝试次数：1] - [剩余数量：1120] - [当前时间：18:15:36]

[o

[ok] - CO10835C
[尝试次数：1] - [剩余数量：995] - [当前时间：18:16:09]

[ok] - CO10842AC
[尝试次数：1] - [剩余数量：994] - [当前时间：18:16:09]

[ok] - CO10837C
[尝试次数：1] - [剩余数量：993] - [当前时间：18:16:10]

[ok] - CO10842PC
[尝试次数：1] - [剩余数量：992] - [当前时间：18:16:10]

[ok] - CO10845AC
[尝试次数：1] - [剩余数量：991] - [当前时间：18:16:11]

[ok] - CO0042C
[尝试次数：2] - [剩余数量：990] - [当前时间：18:16:11]

[ok] - CO10852AC
[尝试次数：1] - [剩余数量：989] - [当前时间：18:16:11]

[ok] - CO10851AC
[尝试次数：1] - [剩余数量：988] - [当前时间：18:16:11]

[ok] - CO10854C
[尝试次数：1] - [剩余数量：987] - [当前时间：18:16:11]

[ok] - CO10855C
[尝试次数：1] - [剩余数量：986] - [当前时间：18:16:12]

[ok] - CO10856C
[尝试次数：1] - [剩余数量：985] - [当前时间：18:16:12]

[ok] - CO10861AC
[尝试次数：1] - [剩余数量：984] - [当前时间：18:16:13]

[ok] - CO10863JC
[尝试次数：1] - [剩余数量：983] - [当前时间：18:16:13]

[ok] - CO10857C
[尝试次数：1] - [剩余数量：982] - [当前时间：18:16:13]

[ok] - CO10864JC
[尝试次数：1] - [剩余数量：981] - [当前时间：18:16:14]

[ok] - CO101640C
[尝试次数：1] - [剩余数量：980] - [当前时间：18:16:14]

[ok] - CO10868C
[尝试次数：1] - [剩余数量：979] - [当前时间：18:16:14]

[ok] - CO10849T
[尝试次数：1

[ok] - CO11036C
[尝试次数：3] - [剩余数量：851] - [当前时间：18:17:09]

[ok] - CO11191C
[尝试次数：1] - [剩余数量：850] - [当前时间：18:17:10]

[ok] - CO11034C
[尝试次数：5] - [剩余数量：849] - [当前时间：18:17:11]

[ok] - CO11192C
[尝试次数：1] - [剩余数量：848] - [当前时间：18:17:11]

[ok] - CO11193C
[尝试次数：1] - [剩余数量：847] - [当前时间：18:17:11]

[ok] - CO11074C
[尝试次数：2] - [剩余数量：846] - [当前时间：18:17:11]

[ok] - CO11194C
[尝试次数：1] - [剩余数量：845] - [当前时间：18:17:12]

[ok] - CO11200C
[尝试次数：1] - [剩余数量：844] - [当前时间：18:17:13]

[ok] - CO11208C
[尝试次数：1] - [剩余数量：843] - [当前时间：18:17:13]

[ok] - CO11209C
[尝试次数：1] - [剩余数量：842] - [当前时间：18:17:15]

[ok] - CO11210C
[尝试次数：1] - [剩余数量：841] - [当前时间：18:17:15]

[ok] - CO11207C
[尝试次数：1] - [剩余数量：840] - [当前时间：18:17:16]

[ok] - CO11211C
[尝试次数：1] - [剩余数量：839] - [当前时间：18:17:16]

[ok] - CO11212C
[尝试次数：1] - [剩余数量：838] - [当前时间：18:17:17]

[ok] - CO11213C
[尝试次数：1] - [剩余数量：837] - [当前时间：18:17:18]

[ok] - CO11063AC
[尝试次数：6] - [剩余数量：836] - [当前时间：18:17:18]

[ok] - CO11217C
[尝试次数：1] - [剩余数量：835] - [当前时间：18:17:19]

[ok] - CO11145C
[尝试次数：3] - [剩余

[ok] - CO11500C
[尝试次数：5] - [剩余数量：707] - [当前时间：18:18:14]

[ok] - CO11509C
[尝试次数：1] - [剩余数量：706] - [当前时间：18:18:15]

[ok] - CO11499C
[尝试次数：1] - [剩余数量：705] - [当前时间：18:18:15]

[ok] - CO11517C
[尝试次数：1] - [剩余数量：704] - [当前时间：18:18:16]

[ok] - CO11510C
[尝试次数：1] - [剩余数量：703] - [当前时间：18:18:17]

[ok] - CO11407C
[尝试次数：1] - [剩余数量：702] - [当前时间：18:18:17]

[ok] - CO11514C
[尝试次数：1] - [剩余数量：701] - [当前时间：18:18:17]

[ok] - CO11531C
[尝试次数：1] - [剩余数量：700] - [当前时间：18:18:18]

[ok] - CO11539C
[尝试次数：1] - [剩余数量：699] - [当前时间：18:18:18]

[ok] - CO11503C
[尝试次数：2] - [剩余数量：698] - [当前时间：18:18:18]

[ok] - CO11529C
[尝试次数：1] - [剩余数量：697] - [当前时间：18:18:20]

[ok] - CO11552C
[尝试次数：1] - [剩余数量：696] - [当前时间：18:18:20]

[ok] - CO11522C
[尝试次数：1] - [剩余数量：695] - [当前时间：18:18:20]

[ok] - CO11560C
[尝试次数：1] - [剩余数量：694] - [当前时间：18:18:20]

[ok] - CO11572C
[尝试次数：1] - [剩余数量：693] - [当前时间：18:18:21]

[ok] - CO11563C
[尝试次数：1] - [剩余数量：692] - [当前时间：18:18:22]

[ok] - CO11578C
[尝试次数：1] - [剩余数量：691] - [当前时间：18:18:22]

[ok] - CO11584C
[尝试次数：1] - [剩余数

[ok] - CO21520C
[尝试次数：1] - [剩余数量：564] - [当前时间：18:19:23]

[ok] - CO21516JC
[尝试次数：7] - [剩余数量：563] - [当前时间：18:19:23]

[ok] - CO21511LC
[尝试次数：11] - [剩余数量：562] - [当前时间：18:19:24]

[ok] - CO21532LC
[尝试次数：8] - [剩余数量：561] - [当前时间：18:19:24]

[ok] - CO21578AC
[尝试次数：2] - [剩余数量：560] - [当前时间：18:19:25]

[ok] - CO21016C
[尝试次数：37] - [剩余数量：559] - [当前时间：18:19:25]

[ok] - CO21193AC
[尝试次数：29] - [剩余数量：558] - [当前时间：18:19:25]

[ok] - CO21603GLC
[尝试次数：1] - [剩余数量：557] - [当前时间：18:19:26]

[ok] - CO21579T
[尝试次数：4] - [剩余数量：556] - [当前时间：18:19:27]

[ok] - CO21625C
[尝试次数：4] - [剩余数量：555] - [当前时间：18:19:27]

[ok] - CO21224C
[尝试次数：33] - [剩余数量：554] - [当前时间：18:19:28]

[ok] - CO21570C
[尝试次数：20] - [剩余数量：553] - [当前时间：18:19:28]

[ok] - CO21676LC
[尝试次数：5] - [剩余数量：552] - [当前时间：18:19:29]

[ok] - CO22002C
[尝试次数：5] - [剩余数量：551] - [当前时间：18:19:30]

[ok] - CO21471LC
[尝试次数：16] - [剩余数量：550] - [当前时间：18:19:30]

[ok] - CO22005C
[尝试次数：1] - [剩余数量：549] - [当前时间：18:19:30]

[ok] - CO22003C
[尝试次数：6] - [剩余数量：548] - [当前时间：18:19:31]

[ok] - CO21584LC

[ok] - CO29111C
[尝试次数：1] - [剩余数量：421] - [当前时间：18:20:13]

[ok] - CO22030C
[尝试次数：19] - [剩余数量：420] - [当前时间：18:20:13]

[ok] - CO29108C
[尝试次数：1] - [剩余数量：419] - [当前时间：18:20:14]

[ok] - CO29113C
[尝试次数：1] - [剩余数量：418] - [当前时间：18:20:14]

[ok] - CO29116C
[尝试次数：1] - [剩余数量：417] - [当前时间：18:20:14]

[ok] - CO29117C
[尝试次数：1] - [剩余数量：416] - [当前时间：18:20:14]

[ok] - CO29114C
[尝试次数：1] - [剩余数量：415] - [当前时间：18:20:15]

[ok] - CO29119C
[尝试次数：1] - [剩余数量：414] - [当前时间：18:20:16]

[ok] - CO29121C
[尝试次数：1] - [剩余数量：413] - [当前时间：18:20:16]

[ok] - CO29123C
[尝试次数：1] - [剩余数量：412] - [当前时间：18:20:16]

[ok] - CO29122C
[尝试次数：1] - [剩余数量：411] - [当前时间：18:20:17]

[ok] - CO29124C
[尝试次数：1] - [剩余数量：410] - [当前时间：18:20:17]

[ok] - CO29125C
[尝试次数：1] - [剩余数量：409] - [当前时间：18:20:17]

[ok] - CO2912GLC
[尝试次数：1] - [剩余数量：408] - [当前时间：18:20:18]

[ok] - CO29127C
[尝试次数：1] - [剩余数量：407] - [当前时间：18:20:19]

[ok] - CO22020C
[尝试次数：22] - [剩余数量：406] - [当前时间：18:20:19]

[ok] - CO22258C
[尝试次数：4] - [剩余数量：405] - [当前时间：18:20:20]

[ok] - CO29133C
[尝试次数：1] - [

[ok] - CO35109C
[尝试次数：1] - [剩余数量：277] - [当前时间：18:20:53]

[ok] - CO3036AC
[尝试次数：1] - [剩余数量：276] - [当前时间：18:20:53]

[ok] - CO35108C
[尝试次数：3] - [剩余数量：275] - [当前时间：18:20:53]

[ok] - CO35112C
[尝试次数：2] - [剩余数量：274] - [当前时间：18:20:53]

[ok] - CO3407AC
[尝试次数：6] - [剩余数量：273] - [当前时间：18:20:54]

[ok] - CO3626AC
[尝试次数：1] - [剩余数量：272] - [当前时间：18:20:54]

[ok] - CO35110C
[尝试次数：2] - [剩余数量：271] - [当前时间：18:20:54]

[ok] - CO30015C
[尝试次数：5] - [剩余数量：270] - [当前时间：18:20:54]

[ok] - CO4036C
[尝试次数：2] - [剩余数量：269] - [当前时间：18:20:55]

[ok] - CO3605AC
[尝试次数：3] - [剩余数量：268] - [当前时间：18:20:55]

[ok] - CO4041SD
[尝试次数：2] - [剩余数量：267] - [当前时间：18:20:55]

[ok] - CO35114C
[尝试次数：3] - [剩余数量：266] - [当前时间：18:20:55]

[ok] - CO4042
[尝试次数：1] - [剩余数量：265] - [当前时间：18:20:56]

[ok] - CO4078C
[尝试次数：1] - [剩余数量：264] - [当前时间：18:20:56]

[ok] - CO35113C
[尝试次数：3] - [剩余数量：263] - [当前时间：18:20:56]

[error] - CO11168MCKTC
[尝试次数：100] - [剩余数量：262] - [当前时间：18:20:56]

[ok] - CO4070C
[尝试次数：3] - [剩余数量：261] - [当前时间：18:20:56]

[ok] - CO4081C
[尝试次数：1] - [

[ok] - CO4637C
[尝试次数：1] - [剩余数量：130] - [当前时间：18:21:34]

[ok] - CO4643C
[尝试次数：1] - [剩余数量：129] - [当前时间：18:21:35]

[ok] - CO4646C
[尝试次数：1] - [剩余数量：128] - [当前时间：18:21:35]

[ok] - CO4643
[尝试次数：1] - [剩余数量：127] - [当前时间：18:21:36]

[ok] - CO4647
[尝试次数：1] - [剩余数量：126] - [当前时间：18:21:36]

[ok] - CO4652C
[尝试次数：1] - [剩余数量：125] - [当前时间：18:21:36]

[ok] - CO4645C
[尝试次数：1] - [剩余数量：124] - [当前时间：18:21:37]

[ok] - CO4660C
[尝试次数：1] - [剩余数量：123] - [当前时间：18:21:37]

[ok] - CO4653C
[尝试次数：1] - [剩余数量：122] - [当前时间：18:21:37]

[ok] - CO46540JC
[尝试次数：1] - [剩余数量：121] - [当前时间：18:21:38]

[ok] - CO42038V
[尝试次数：3] - [剩余数量：120] - [当前时间：18:21:38]

[ok] - CO4663C
[尝试次数：1] - [剩余数量：119] - [当前时间：18:21:38]

[ok] - CO4672C
[尝试次数：1] - [剩余数量：118] - [当前时间：18:21:39]

[ok] - CO4687C
[尝试次数：1] - [剩余数量：117] - [当前时间：18:21:39]

[ok] - CO4667C
[尝试次数：1] - [剩余数量：116] - [当前时间：18:21:40]

[ok] - CO4688C
[尝试次数：1] - [剩余数量：115] - [当前时间：18:21:41]

[ok] - CO4695C
[尝试次数：1] - [剩余数量：114] - [当前时间：18:21:41]

[ok] - CO4691C
[尝试次数：1] - [剩余数量：113] - [当前时间：18

[ok] - COYC180C
[尝试次数：4] - [剩余数量：0] - [当前时间：18:23:06]

输出ing...

爬虫存在error

Done ~
